In [34]:
import numpy as np
import pandas as pd
import csv
import re

# create GO-list tab delimeted  from .obo format

In [75]:
file = open("GOlist.obo", "r")

In [76]:
GO_df = pd.DataFrame()
_id = ""
name = ""
namespace = ""
_def = ""
synonym = []
_term = False

In [77]:
for line in file:

    if line == "\n":
        if _term == False:    
            pass
        elif _term == True:
            # save the previous one
            GO_df = GO_df.append(pd.DataFrame({'ID': _id,'name':name , "namespace":namespace ,"definition" : _def}, index=[0]), ignore_index=True)
            # start the new record
            _id = ""
            name= ""
            namespace = ""
            _def = ""
            _term = False
        continue
        
    line = line.replace('\n', '')
    
    if line.startswith('[Term]'):
        _term = True
              
    if line.startswith('id:'):
        x = line.split()
        _id = x[1] # get id 
        
    if line.startswith('name:'):
        x = line.split(" ", 1)
        name = x[1] # get name
    
    if line.startswith('namespace:'):
        x = line.split(" ", 1)
        namespace = x[1] # get namespace
    
    if line.startswith('def:'):
        x = re.findall(r'"([^"]*)"', line)
        _def = x[0] # get def
    
    #Keep the last one
   # if line.startswith('[Typedef]'):
   #     GO_df = GO_df.append(pd.DataFrame({'ID': _id,'name':name , "name_sapce":namespace ,"definition" : _def}, index=[0]), ignore_index=True)
        

In [78]:
# GO_df.loc[4]["definition"]
GO_df

,ID,name,namespace,definition
0,GO:0000001,mitochondrion inheritance,biological_process,"The distribution of mitochondria, including th..."
1,GO:0000002,mitochondrial genome maintenance,biological_process,The maintenance of the structure and integrity...
2,GO:0000003,reproduction,biological_process,The production of new individuals that contain...
3,GO:0000005,obsolete ribosomal chaperone activity,molecular_function,OBSOLETE. Assists in the correct assembly of r...
4,GO:0000006,high-affinity zinc transmembrane transporter a...,molecular_function,Enables the transfer of zinc ions (Zn2+) from ...
...,...,...,...,...
47205,GO:2001313,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,The chemical reactions and pathways involving ...
47206,GO:2001314,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,The chemical reactions and pathways resulting ...
47207,GO:2001315,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,The chemical reactions and pathways resulting ...
47208,GO:2001316,kojic acid metabolic process,biological_process,The chemical reactions and pathways involving ...


In [81]:
GO_df.to_csv("GOTerms.csv", columns=GO_df.columns ,sep=",", quotechar='"',index=False,
            quoting=csv.QUOTE_ALL)

## Create AMP GO term

In [138]:
## Find GO Id that contains antibacterial,antimicrobial, antifungal, antiviral, defense, immun,  anticancer
_tmp = GO_df[GO_df['definition'].str.contains("antibacterial|antimicrobial|antifungal|antiviral|defense|defence|immune|immunity|immunological|anticancer")]
_tmp

,ID,name,namespace,definition
1085,GO:0001776,leukocyte homeostasis,biological_process,The process of regulating the proliferation an...
1092,GO:0001783,B cell apoptotic process,biological_process,"Any apoptotic process in a B cell, a lymphocyt..."
1110,GO:0001802,type III hypersensitivity,biological_process,An inflammatory response resulting from recogn...
1128,GO:0001820,serotonin secretion,biological_process,The regulated release of serotonin by a cell. ...
1154,GO:0001846,opsonin binding,molecular_function,Interacting selectively and non-covalently wit...
...,...,...,...,...
46440,GO:2000522,positive regulation of immunological synapse f...,biological_process,Any process that activates or increases the fr...
46444,GO:2000526,positive regulation of glycoprotein biosynthet...,biological_process,Any positive regulation of glycoprotein biosyn...
47086,GO:2001191,regulation of gamma-delta T cell activation in...,biological_process,"Any process that modulates the frequency, rate..."
47087,GO:2001192,negative regulation of gamma-delta T cell acti...,biological_process,"Any process that stops, prevents or reduces th..."


In [89]:
# _tmp = GO_df[GO_df['definition'].str.contains("anti|immun|defense")] <-- incase create  super terms

In [139]:
_tmp.to_csv("amp_tmp.csv", columns=GO_df.columns ,sep=",", quotechar='"',index=False,
            quoting=csv.QUOTE_ALL)

# Check with groud truth

In [140]:
amp_go_df = pd.read_csv('amp_tmp.csv', sep=",", header=0,quoting=csv.QUOTE_ALL)  
amp_go_df

,ID,name,namespace,definition
0,GO:0001776,leukocyte homeostasis,biological_process,The process of regulating the proliferation an...
1,GO:0001783,B cell apoptotic process,biological_process,"Any apoptotic process in a B cell, a lymphocyt..."
2,GO:0001802,type III hypersensitivity,biological_process,An inflammatory response resulting from recogn...
3,GO:0001820,serotonin secretion,biological_process,The regulated release of serotonin by a cell. ...
4,GO:0001846,opsonin binding,molecular_function,Interacting selectively and non-covalently wit...
...,...,...,...,...
672,GO:2000522,positive regulation of immunological synapse f...,biological_process,Any process that activates or increases the fr...
673,GO:2000526,positive regulation of glycoprotein biosynthet...,biological_process,Any positive regulation of glycoprotein biosyn...
674,GO:2001191,regulation of gamma-delta T cell activation in...,biological_process,"Any process that modulates the frequency, rate..."
675,GO:2001192,negative regulation of gamma-delta T cell acti...,biological_process,"Any process that stops, prevents or reduces th..."


## Bat

In [141]:
anno_BAT = pd.read_csv('BAT_annotation_report.txt', sep="\t", header=0,quoting=csv.QUOTE_ALL)  
anno_BAT

,#gene_id,transcript_id,sprot_Top_BLASTX_hit,RNAMMER,prot_id,prot_coords,sprot_Top_BLASTP_hit,uniprot_metazoa_33208.fasta_BLASTX,uniprot_metazoa_33208.fasta_BLASTP,Pfam,SignalP,TmHMM,eggnog,Kegg,gene_ontology_BLASTX,gene_ontology_BLASTP,gene_ontology_Pfam,transcript,peptide
0,TRINITY_DN37504_c0_g1_i1,TRINITY_DN37504_c0_g1_i1,"PELI1_HUMAN^PELI1_HUMAN^Q:251-111,H:26-72^93.6...",.,.,.,.,sp|Q8C669|PELI1_MOUSE^sp|Q8C669|PELI1_MOUSE^Q:...,.,.,.,.,.,KEGG:hsa:57162,GO:0000778^cellular_component^condensed nuclea...,.,.,.,.
1,TRINITY_DN37521_c0_g1_i1,TRINITY_DN37521_c0_g1_i1,"YAF2_HUMAN^YAF2_HUMAN^Q:15-167,H:1-51^100%ID^E...",.,.,.,.,tr|A0A3Q2HRA4|A0A3Q2HRA4_HORSE^tr|A0A3Q2HRA4|A...,.,.,.,.,.,KEGG:hsa:10138,GO:0005829^cellular_component^cytosol`GO:00056...,.,.,.,.
2,TRINITY_DN37522_c0_g1_i1,TRINITY_DN37522_c0_g1_i1,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
3,TRINITY_DN18635_c0_g1_i1,TRINITY_DN18635_c0_g1_i1,"SPTB2_HUMAN^SPTB2_HUMAN^Q:216-82,H:1488-1532^8...",.,.,.,.,tr|A0A6I9HUA4|A0A6I9HUA4_GEOFO^tr|A0A6I9HUA4|A...,.,.,.,.,.,KEGG:hsa:6711,GO:0030673^cellular_component^axolemma`GO:0005...,.,.,.,.
4,TRINITY_DN18683_c0_g1_i1,TRINITY_DN18683_c0_g1_i1,.,.,TRINITY_DN18683_c0_g1_i1.p1,2-313[+],.,tr|A0A3P4RZH0|A0A3P4RZH0_GULGU^tr|A0A3P4RZH0|A...,.,.,.,.,.,.,.,.,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21337,SOAP.k61.C41381,SOAP.k61.C41381,"TKT_MOUSE^TKT_MOUSE^Q:3-1868,H:1-622^94.2%ID^E...",.,SOAP.k61.C41381.p1,3-1874[+],"TKT_MOUSE^TKT_MOUSE^Q:1-622,H:1-622^94.2%ID^E:...",tr|G1NUG7|G1NUG7_MYOLU^tr|G1NUG7|G1NUG7_MYOLU^...,tr|G1NUG7|G1NUG7_MYOLU^tr|G1NUG7|G1NUG7_MYOLU^...,"PF00456.22^Transketolase_N^Transketolase, thia...",.,.,.,KEGG:mmu:21881,GO:0005789^cellular_component^endoplasmic reti...,GO:0005789^cellular_component^endoplasmic reti...,GO:0008661^molecular_function^1-deoxy-D-xylulo...,.,.
21338,SOAP.k61.C41407,SOAP.k61.C41407,"DNJB2_HUMAN^DNJB2_HUMAN^Q:158-1132,H:1-324^79....",.,SOAP.k61.C41407.p1,158-1135[+],"DNJB2_HUMAN^DNJB2_HUMAN^Q:1-325,H:1-324^79.4%I...",tr|G1PAR2|G1PAR2_MYOLU^tr|G1PAR2|G1PAR2_MYOLU^...,tr|G1PAR2|G1PAR2_MYOLU^tr|G1PAR2|G1PAR2_MYOLU^...,PF00226.32^DnaJ^DnaJ domain^4-66^E:1.1e-26`PF0...,.,.,.,KEGG:hsa:3300,GO:0005737^cellular_component^cytoplasm`GO:009...,GO:0005737^cellular_component^cytoplasm`GO:009...,.,.,.
21339,SOAP.k61.C41407,SOAP.k61.C41407,"DNJB2_HUMAN^DNJB2_HUMAN^Q:158-1132,H:1-324^79....",.,SOAP.k61.C41407.p3,1340-1648[+],.,tr|G1PAR2|G1PAR2_MYOLU^tr|G1PAR2|G1PAR2_MYOLU^...,.,.,.,.,.,KEGG:hsa:3300,GO:0005737^cellular_component^cytoplasm`GO:009...,.,.,.,.
21340,SOAP.k61.C41445,SOAP.k61.C41445,"KAT2B_HUMAN^KAT2B_HUMAN^Q:2381-186,H:101-832^9...",.,SOAP.k61.C41445.p1,183-2300[-],"KAT2B_HUMAN^KAT2B_HUMAN^Q:1-705,H:128-832^97.2...",tr|S7P7K2|S7P7K2_MYOBR^tr|S7P7K2|S7P7K2_MYOBR^...,tr|S7P7K2|S7P7K2_MYOBR^tr|S7P7K2|S7P7K2_MYOBR^...,PF06466.12^PCAF_N^PCAF (P300/CBP-associated fa...,.,.,.,KEGG:hsa:8850,GO:0031672^cellular_component^A band`GO:004264...,GO:0031672^cellular_component^A band`GO:004264...,GO:0004402^molecular_function^histone acetyltr...,.,.


In [161]:
anno_BAT['all_GO'] = anno_BAT['gene_ontology_BLASTX'] + '`' + anno_BAT['gene_ontology_BLASTP'] + "`" +  anno_BAT['gene_ontology_Pfam']
BAT_truth_set = anno_BAT[ anno_BAT.all_GO.str.contains ('|'.join(amp_go_df.ID.to_list()))]
BAT_truth_set = BAT_truth_set[BAT_truth_set["prot_id"] != "."]
BAT_truth_set

,#gene_id,transcript_id,sprot_Top_BLASTX_hit,RNAMMER,prot_id,prot_coords,sprot_Top_BLASTP_hit,uniprot_metazoa_33208.fasta_BLASTX,uniprot_metazoa_33208.fasta_BLASTP,Pfam,SignalP,TmHMM,eggnog,Kegg,gene_ontology_BLASTX,gene_ontology_BLASTP,gene_ontology_Pfam,transcript,peptide,all_GO
7,TRINITY_DN18674_c0_g1_i1,TRINITY_DN18674_c0_g1_i1,"TRAD1_BOVIN^TRAD1_BOVIN^Q:124-738,H:1-204^87.3...",.,TRINITY_DN18674_c0_g1_i1.p1,124-741[+],"TRAD1_BOVIN^TRAD1_BOVIN^Q:1-205,H:1-204^87.3%I...",tr|L5LJI1|L5LJI1_MYODS^tr|L5LJI1|L5LJI1_MYODS^...,tr|L5LJI1|L5LJI1_MYODS^tr|L5LJI1|L5LJI1_MYODS^...,.,.,.,.,KEGG:bta:512642,GO:0005739^cellular_component^mitochondrion`GO...,GO:0005739^cellular_component^mitochondrion`GO...,.,.,.,GO:0005739^cellular_component^mitochondrion`GO...
9,TRINITY_DN18675_c0_g1_i1,TRINITY_DN18675_c0_g1_i1,"IRF1_BOVIN^IRF1_BOVIN^Q:399-214,H:1-62^98.4%ID...",.,TRINITY_DN18675_c0_g1_i1.p1,1-399[-],"IRF1_BOVIN^IRF1_BOVIN^Q:1-62,H:1-62^98.4%ID^E:...",tr|A0A673U5G9|A0A673U5G9_SURSU^tr|A0A673U5G9|A...,tr|A0A384C647|A0A384C647_URSMA^tr|A0A384C647|A...,PF00605.18^IRF^Interferon regulatory factor tr...,.,.,.,KEGG:bta:789216,GO:0005737^cellular_component^cytoplasm`GO:000...,GO:0005737^cellular_component^cytoplasm`GO:000...,.,.,.,GO:0005737^cellular_component^cytoplasm`GO:000...
19,TRINITY_DN34002_c0_g1_i1,TRINITY_DN34002_c0_g1_i1,"ACKR1_SAGIM^ACKR1_SAGIM^Q:1021-41,H:1-336^63.2...",.,TRINITY_DN34002_c0_g1_i1.p1,38-1021[-],"ACKR1_SAGIM^ACKR1_SAGIM^Q:1-327,H:1-336^63.2%I...",tr|S7NNL5|S7NNL5_MYOBR^tr|S7NNL5|S7NNL5_MYOBR^...,tr|S7NNL5|S7NNL5_MYOBR^tr|S7NNL5|S7NNL5_MYOBR^...,.,.,.,.,.,GO:0005769^cellular_component^early endosome`G...,GO:0005769^cellular_component^early endosome`G...,.,.,.,GO:0005769^cellular_component^early endosome`G...
34,TRINITY_DN4863_c0_g1_i1,TRINITY_DN4863_c0_g1_i1,"NECT1_PIG^NECT1_PIG^Q:265-1074,H:15-284^85.9%I...",.,TRINITY_DN4863_c0_g1_i1.p4,1088-1750[+],"NECT1_HUMAN^NECT1_HUMAN^Q:1-221,H:32-252^87.3%...",tr|G1NT63|G1NT63_MYOLU^tr|G1NT63|G1NT63_MYOLU^...,tr|L5M6Q8|L5M6Q8_MYODS^tr|L5M6Q8|L5M6Q8_MYODS^...,PF00047.26^ig^Immunoglobulin domain^5-99^E:1.1...,.,.,.,KEGG:hsa:5818,GO:0005912^cellular_component^adherens junctio...,GO:0005912^cellular_component^adherens junctio...,.,.,.,GO:0005912^cellular_component^adherens junctio...
64,TRINITY_DN4849_c0_g1_i2,TRINITY_DN4849_c0_g1_i2,"ZBT7A_HUMAN^ZBT7A_HUMAN^Q:410-3,H:1-136^99.3%I...",.,TRINITY_DN4849_c0_g1_i2.p1,3-455[-],"ZBT7A_HUMAN^ZBT7A_HUMAN^Q:16-151,H:1-136^99.3%...",tr|A0A3Q2KL94|A0A3Q2KL94_HORSE^tr|A0A3Q2KL94|A...,tr|A0A3Q2KL94|A0A3Q2KL94_HORSE^tr|A0A3Q2KL94|A...,PF00651.32^BTB^BTB/POZ domain^39-145^E:1.8e-33,.,.,.,KEGG:hsa:51341,GO:0005737^cellular_component^cytoplasm`GO:007...,GO:0005737^cellular_component^cytoplasm`GO:007...,GO:0005515^molecular_function^protein binding,.,.,GO:0005737^cellular_component^cytoplasm`GO:007...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21174,SOAP.k61.scaffold787,SOAP.k61.scaffold787,"TRI13_HUMAN^TRI13_HUMAN^Q:1292-75,H:1-407^91.6...",.,SOAP.k61.scaffold787.p2,1360-1686[-],.,tr|G1Q7V7|G1Q7V7_MYOLU^tr|G1Q7V7|G1Q7V7_MYOLU^...,.,.,.,.,.,KEGG:hsa:10206,GO:0005737^cellular_component^cytoplasm`GO:000...,.,.,.,.,GO:0005737^cellular_component^cytoplasm`GO:000...
21175,SOAP.k61.scaffold787,SOAP.k61.scaffold787,"TRI13_HUMAN^TRI13_HUMAN^Q:1292-75,H:1-407^91.6...",.,SOAP.k61.scaffold787.p1,72-1292[-],"TRI13_HUMAN^TRI13_HUMAN^Q:1-406,H:1-407^91.6%I...",tr|G1Q7V7|G1Q7V7_MYOLU^tr|G1Q7V7|G1Q7V7_MYOLU^...,tr|G1Q7V7|G1Q7V7_MYOLU^tr|G1Q7V7|G1Q7V7_MYOLU^...,"PF13920.7^zf-C3HC4_3^Zinc finger, C3HC4 type (...",.,.,.,KEGG:hsa:10206,GO:0005737^cellular_component^cytoplasm`GO:000...,GO:0005737^cellular_component^cytoplasm`GO:000...,GO:0046872^molecular_function^metal ion bindin...,.,.,GO:0005737^cellular_component^cytoplasm`GO:000...
21296,SOAP.k61.C37063,SOAP.k61.C37063,"FAK1_HUMAN^FAK1_HUMAN^Q:339-1,H:678-790^89.4%I...",.,SOAP.k61.C37063.p2,206-520[-],"FAK1_CHICK^FAK1_CHICK^Q:1-60,H:774-833^81.7%ID...",tr|L5MIB4|L5MIB4_MYODS

In [162]:
BAT_truth_set.to_csv("BAT_truth_set.csv", columns=BAT_truth_set.columns ,sep=",", quotechar='"',index=False,
            quoting=csv.QUOTE_ALL)

## DECockroach

In [148]:
anno_DECockroach = pd.read_csv('DECockroach_ann_report.txt', sep="\t", header=0,quoting=csv.QUOTE_ALL)  
anno_DECockroach

,#gene_id,transcript_id,sprot_Top_BLASTX_hit,RNAMMER,prot_id,prot_coords,sprot_Top_BLASTP_hit,uniprot_metazoa_33208.fasta_BLASTX,uniprot_metazoa_33208.fasta_BLASTP,Pfam,SignalP,TmHMM,eggnog,Kegg,gene_ontology_BLASTX,gene_ontology_BLASTP,gene_ontology_Pfam,transcript,peptide
0,TRINITY_DN18621_c2_g1_i1,TRINITY_DN18621_c2_g1_i1,.,.,TRINITY_DN18621_c2_g1_i1.p1,144-689[-],.,tr|A0A2P8XSV6|A0A2P8XSV6_BLAGE^tr|A0A2P8XSV6|A...,tr|A0A2P8XJ83|A0A2P8XJ83_BLAGE^tr|A0A2P8XJ83|A...,.,.,.,.,.,.,.,.,.,.
1,TRINITY_DN18621_c2_g2_i1,TRINITY_DN18621_c2_g2_i1,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
2,TRINITY_DN18657_c0_g1_i1,TRINITY_DN18657_c0_g1_i1,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
3,TRINITY_DN18694_c0_g1_i1,TRINITY_DN18694_c0_g1_i1,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
4,TRINITY_DN18694_c0_g2_i1,TRINITY_DN18694_c0_g2_i1,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55044,SOAP.k37.C212152,SOAP.k37.C212152,"P52K_HUMAN^P52K_HUMAN^Q:1963-227,H:163-740^27....",.,SOAP.k37.C212152.p1,197-2713[-],"P52K_HUMAN^P52K_HUMAN^Q:251-829,H:163-740^27.9...",tr|A0A2P8YUA9|A0A2P8YUA9_BLAGE^tr|A0A2P8YUA9|A...,tr|A0A2P8YUA9|A0A2P8YUA9_BLAGE^tr|A0A2P8YUA9|A...,PF14291.7^DUF4371^Domain of unknown function (...,.,.,.,KEGG:hsa:5612,GO:0003677^molecular_function^DNA binding`GO:0...,GO:0003677^molecular_function^DNA binding`GO:0...,GO:0046983^molecular_function^protein dimeriza...,.,.
55045,SOAP.k37.C212262,SOAP.k37.C212262,"STAUH_APLCA^STAUH_APLCA^Q:2779-758,H:336-944^4...",.,SOAP.k37.C212262.p1,704-3211[-],"STAUH_APLCA^STAUH_APLCA^Q:145-818,H:336-944^46...",tr|A0A2J7QF72|A0A2J7QF72_9NEOP^tr|A0A2J7QF72|A...,tr|A0A2J7QF72|A0A2J7QF72_9NEOP^tr|A0A2J7QF72|A...,PF00035.27^dsrm^Double-stranded RNA binding mo...,.,.,.,.,GO:0043005^cellular_component^neuron projectio...,GO:0043005^cellular_component^neuron projectio...,.,.,.
55046,SOAP.k37.C212368,SOAP.k37.C212368,"CTL12_ANOGA^CTL12_ANOGA^Q:3308-1233,H:26-790^4...",.,SOAP.k37.C212368.p1,1230-3350[-],"CTL12_ANOGA^CTL12_ANOGA^Q:15-706,H:26-790^41.1...",tr|A0A2P8XXU7|A0A2P8XXU7_BLAGE^tr|A0A2P8XXU7|A...,tr|A0A2P8XXU7|A0A2P8XXU7_BLAGE^tr|A0A2P8XXU7|A...,PF04515.13^Choline_transpo^Plasma-membrane cho...,.,.,.,KEGG:aga:AgaP_AGAP010343,GO:0016021^cellular_component^integral compone...,GO:0016021^cellular_component^integral compone...,.,.,.
55047,SOAP.k37.C212440,SOAP.k37.C212440,"PPIF_HUMAN^PPIF_HUMAN^Q:1536-2024,H:46-206^41....",.,SOAP.k37.C212440.p1,264-2027[+],"PPIF_HUMAN^PPIF_HUMAN^Q:425-587,H:46-206^41.6%...",tr|A0A2P8XPB5|A0A2P8XPB5_BLAGE^tr|A0A2P8XPB5|A...,tr|A0A2P8XPB5|A0A2P8XPB5_BLAGE^tr|A0A2P8XPB5|A...,PF00643.25^zf-B_box^B-box zinc finger^112-149^...,.,.,.,KEGG:hsa:10105,GO:0005737^cellular_component^cytoplasm`GO:004...,GO:0005737^cellular_component^cytoplasm`GO:004...,GO:0008270^molecular_function^zinc ion binding...,.,.


In [159]:
anno_DECockroach['all_GO'] = anno_DECockroach['gene_ontology_BLASTX'] + '`' + anno_DECockroach['gene_ontology_BLASTP'] + "`" +  anno_DECockroach['gene_ontology_Pfam']
DECockroach_truth_set = anno_DECockroach[ anno_DECockroach.all_GO.str.contains ('|'.join(amp_go_df.ID.to_list()))]
DECockroach_truth_set = DECockroach_truth_set[DECockroach_truth_set["prot_id"] != "."]
DECockroach_truth_set

,#gene_id,transcript_id,sprot_Top_BLASTX_hit,RNAMMER,prot_id,prot_coords,sprot_Top_BLASTP_hit,uniprot_metazoa_33208.fasta_BLASTX,uniprot_metazoa_33208.fasta_BLASTP,Pfam,SignalP,TmHMM,eggnog,Kegg,gene_ontology_BLASTX,gene_ontology_BLASTP,gene_ontology_Pfam,transcript,peptide,all_GO
87,TRINITY_DN18302_c0_g3_i1,TRINITY_DN18302_c0_g3_i1,"LAMB1_DROME^LAMB1_DROME^Q:147-5474,H:5-1786^50...",.,TRINITY_DN18302_c0_g3_i1.p1,123-5483[+],"LAMB1_DROME^LAMB1_DROME^Q:9-1784,H:5-1786^50.5...",tr|A0A1V1FV97|A0A1V1FV97_9NEOP^tr|A0A1V1FV97|A...,tr|A0A1V1FV97|A0A1V1FV97_9NEOP^tr|A0A1V1FV97|A...,PF00055.18^Laminin_N^Laminin N-terminal (Domai...,.,.,.,KEGG:dme:Dmel_CG7123,GO:0005604^cellular_component^basement membran...,GO:0005604^cellular_component^basement membran...,.,.,.,GO:0005604^cellular_component^basement membran...
317,TRINITY_DN21693_c0_g1_i1,TRINITY_DN21693_c0_g1_i1,"FAT2_DROME^FAT2_DROME^Q:312-19,H:3528-3625^66....",.,TRINITY_DN21693_c0_g1_i1.p1,1-312[-],"FAT2_DROME^FAT2_DROME^Q:1-98,H:3528-3625^66.3%...",tr|A0A2J7Q2R9|A0A2J7Q2R9_9NEOP^tr|A0A2J7Q2R9|A...,tr|A0A2J7Q2R9|A0A2J7Q2R9_9NEOP^tr|A0A2J7Q2R9|A...,.,.,.,.,KEGG:dme:Dmel_CG7749,GO:0098858^cellular_component^actin-based cell...,GO:0098858^cellular_component^actin-based cell...,.,.,.,GO:0098858^cellular_component^actin-based cell...
468,TRINITY_DN3762_c0_g1_i3,TRINITY_DN3762_c0_g1_i3,"TTLL1_BOVIN^TTLL1_BOVIN^Q:1404-253,H:4-387^68....",.,TRINITY_DN3762_c0_g1_i3.p1,196-1407[-],"TTLL1_BOVIN^TTLL1_BOVIN^Q:2-385,H:4-387^68.1%I...",tr|A0A6L2PJ95|A0A6L2PJ95_COPFO^tr|A0A6L2PJ95|A...,tr|A0A6L2PJ95|A0A6L2PJ95_COPFO^tr|A0A6L2PJ95|A...,PF03133.16^TTL^Tubulin-tyrosine ligase family^...,.,.,.,KEGG:bta:539530,GO:0036064^cellular_component^ciliary basal bo...,GO:0036064^cellular_component^ciliary basal bo...,GO:0006464^biological_process^cellular protein...,.,.,GO:0036064^cellular_component^ciliary basal bo...
521,TRINITY_DN24079_c0_g1_i1,TRINITY_DN24079_c0_g1_i1,"NU301_DROME^NU301_DROME^Q:12-353,H:615-728^56....",.,TRINITY_DN24079_c0_g1_i1.p1,3-353[+],"NU301_DROME^NU301_DROME^Q:4-117,H:615-728^56.1...",tr|A0A2J7RD28|A0A2J7RD28_9NEOP^tr|A0A2J7RD28|A...,tr|A0A2J7RD28|A0A2J7RD28_9NEOP^tr|A0A2J7RD28|A...,.,.,.,.,KEGG:dme:Dmel_CG32346,GO:0005634^cellular_component^nucleus`GO:00165...,GO:0005634^cellular_component^nucleus`GO:00165...,.,.,.,GO:0005634^cellular_component^nucleus`GO:00165...
539,TRINITY_DN5073_c0_g2_i1,TRINITY_DN5073_c0_g2_i1,"SPON2_RAT^SPON2_RAT^Q:73-750,H:16-245^43.9%ID^...",.,TRINITY_DN5073_c0_g2_i1.p1,25-1359[+],"SPON2_RAT^SPON2_RAT^Q:17-242,H:16-245^43.9%ID^...",tr|A0A2J7Q4H2|A0A2J7Q4H2_9NEOP^tr|A0A2J7Q4H2|A...,tr|A0A2J7Q4H2|A0A2J7Q4H2_9NEOP^tr|A0A2J7Q4H2|A...,PF06468.14^Spond_N^Spondin_N^42-231^E:8.8e-73`...,.,.,.,KEGG:rno:100910790`KEGG:rno:171569,GO:0031012^cellular_component^extracellular ma...,GO:0031012^cellular_component^extracellular ma...,.,.,.,GO:0031012^cellular_component^extracellular ma...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54000,SOAP.k33.C294190,SOAP.k33.C294190,"G3BP2_MOUSE^G3BP2_MOUSE^Q:189-2099,H:1-476^33%...",.,SOAP.k33.C294190.p1,189-2135[+],"G3BP2_MOUSE^G3BP2_MOUSE^Q:1-637,H:1-476^33%ID^...",tr|A0A2P8XIY8|A0A2P8XIY8_BLAGE^tr|A0A2P8XIY8|A...,tr|A0A2P8XIY8|A0A2P8XIY8_BLAGE^tr|A0A2P8XIY8|A...,PF02136.21^NTF2^Nuclear transport factor 2 (NT...,.,.,.,KEGG:mmu:23881,GO:0010494^cellular_component^cytoplasmic stre...,GO:0010494^cellular_component^cytoplasmic stre...,GO:0003676^molecular_function^nucleic acid bin...,.,.,GO:0010494^cellular_component^cytoplasmic stre...
54942,SOAP.k37.C205857,SOAP.k37.C205857,"SPZ5_DROME^SPZ5_DROME^Q:138-971,H:32-387^32.7%...",.,SOAP.k37.C205857.p1,75-980[+],"SPZ5_DROME^SPZ5_DROME^Q:22-299,H:32-387^32.7%I...",tr|A0A6L2PQP7|A0A6L2PQP7_COPFO^tr|A0A6L2PQP7|A...,tr|A0A6L2PQP7|A0A6L2PQP7_COPFO^tr|A0A6L2PQP7|A...,PF16077.6^Spaetzle^Spaetzle^200-296^E:4.6e-28,.,.,.,KEGG:dme:Dmel_CG9972,GO:0005576^cellular_component^extracellular re...,GO:0005576^cellular_component^extracellular re...,.,.,.,GO:0005576^ce

In [160]:
DECockroach_truth_set.to_csv("DECockroach_truth_set.csv", columns=BAT_truth_set.columns ,sep=",", quotechar='"',index=False,
            quoting=csv.QUOTE_ALL)

## Search Term 

In [187]:
string_tmp =  BAT_truth_set[BAT_truth_set["prot_id"] == "SOAP.k25.C309190.p1"]["gene_ontology_BLASTX"].values[0]

In [188]:
string_tmp

'GO:0070419^cellular_component^nonhomologous end joining complex`GO:0005730^cellular_component^nucleolus`GO:0005634^cellular_component^nucleus`GO:0032040^cellular_component^small-subunit processome`GO:0005524^molecular_function^ATP binding`GO:0003677^molecular_function^DNA binding`GO:0004677^molecular_function^DNA-dependent protein kinase activity`GO:0106310^molecular_function^protein serine kinase activity`GO:0004674^molecular_function^protein serine/threonine kinase activity`GO:0106311^molecular_function^protein threonine kinase activity`GO:0034511^molecular_function^U3 snoRNA binding`GO:0006974^biological_process^cellular response to DNA damage stimulus`GO:0006302^biological_process^double-strand break repair`GO:0006303^biological_process^double-strand break repair via nonhomologous end joining`GO:0033152^biological_process^immunoglobulin V(D)J recombination`GO:0045087^biological_process^innate immune response`GO:0008630^biological_process^intrinsic apoptotic signaling pathway in re

In [195]:
for go_term in amp_go_df.ID.to_list():
    position = string_tmp.find(go_term)
    if(position != -1):
        print(go_term)


GO:0045087


In [199]:
amp_go_df[amp_go_df["ID"] == "GO:0045087"]["definition"].values

array(['Innate immune responses are defense responses mediated by germline encoded components that directly recognize components of potential pathogens.'],
      dtype=object)